In [10]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [11]:
!git clone https://github.com/jahidhasan17/ML-Assignment.git

fatal: destination path 'ML-Assignment' already exists and is not an empty directory.


In [12]:
data_dir = 'ML-Assignment/CNN_dataset/'
categories = ['AbdomenCT', 'HeadCT', 'ChestCT', 'Hand', 'CXR']

In [13]:
images = []
labels = []
class_names = ['HeadCT', 'ChestCT', 'AbdomenCT', 'Hand', 'CXR']


for i, class_name in enumerate(class_names):
    folder_path = os.path.join(data_dir, class_name)
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (32, 32))
        images.append(img)
        labels.append(class_names.index(class_name))


In [14]:
# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [15]:
# Normalize pixel values
images = images.astype('float32') / 255.0

In [22]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.3, random_state=42)

In [23]:
# Balance the dataset
class_counts = np.bincount(train_labels)
max_count = np.max(class_counts)
for i, count in enumerate(class_counts):
    if count != max_count:
        mask = train_labels == i
        train_images = np.concatenate((train_images, train_images[mask]), axis=0)
        train_labels = np.concatenate((train_labels, train_labels[mask]), axis=0)

In [25]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

In [26]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
history = model.fit(train_images, train_labels, epochs=5, batch_size=32, validation_split=0.1)


Epoch 1/5
1772/1772 [==============================] - 93s 52ms/step - loss: 0.0785 - accuracy: 0.9743 - val_loss: 0.0228 - val_accuracy: 0.9930
Epoch 2/5
1772/1772 [==============================] - 89s 50ms/step - loss: 0.0173 - accuracy: 0.9953 - val_loss: 0.0120 - val_accuracy: 0.9965
Epoch 3/5
1772/1772 [==============================] - 90s 51ms/step - loss: 0.0100 - accuracy: 0.9973 - val_loss: 0.0110 - val_accuracy: 0.9954
Epoch 4/5
1772/1772 [==============================] - 87s 49ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.0047 - val_accuracy: 0.9989
Epoch 5/5
1772/1772 [==============================] - 91s 51ms/step - loss: 0.0046 - accuracy: 0.9986 - val_loss: 4.7335e-04 - val_accuracy: 0.9998


In [28]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}')

469/469 [==============================] - 7s 14ms/step - loss: 0.0026 - accuracy: 0.9993
Test loss: 0.0026, Test accuracy: 0.9993
